In [90]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [91]:
from data import conll_to_csv
from pathlib import Path
import pandas as pd

## Get Data

download the conll-2003 dataset
for example from https://github.com/Nidhi-K/Natural-Language-Processing-Projects/tree/master/Sequential%20CRF%20for%20NER/data

and save it to DATA_PATH

In [94]:
DATA_PATH = Path('./data/conll-2003/test')
DATA_PATH.mkdir(parents=True, exist_ok=True)

In [102]:
! svn co 'https://github.com/Nidhi-K/Natural-Language-Processing-Projects/trunk/Sequential CRF for NER/data/' {DATA_PATH}
# rename testb for consistency
! mv {DATA_PATH}/eng.testb.blind {DATA_PATH}/eng.testb

Checked out revision 32.


In [105]:
files =['eng.train', 'eng.testa', 'eng.testb']
x = [DATA_PATH/file for file in files]
conll_to_csv('./csv/conll-2003/csv',x)

Wrote csv/conll-2003/csv/eng.train.csv
Wrote csv/conll-2003/csv/eng.testa.csv
Wrote csv/conll-2003/csv/eng.testb.csv


## Create DataBunch